# Machball: a very simple tutorial

This is a simple tutorial on how to use and how to make the most of Machball. It assumes that you have a Python distribution with the jupyter environment, numpy (which is one of Machball's dependencies), matplotlib, and Machball installed.

Let's start by importing matplotlib and numpy:

In [ ]:
import matplotlib.pyplot as pt
import numpy as np

These packages will allow us to work with arrays and matrices as well as to plot the results:

In [ ]:
x = np.arange(0,20,0.1)
y = np.sin(x)
pt.plot(x,y)

## Defining an ALD process in Machball

Machball defines an ideal ALD process using the ```ALDIdeal``` class. It implements a first order irreversible Langmuir kinetics. Key parameters are the precursor pressure (in Pa), molecular mass (in amu or grams/mole), the bare or initial sticking probability (the reactivity of a precursor molecule towards an available surface site), the temperature of the process (in K) and the average area of a surface site (in square meters).

### Defining process parameters

We can start by defining some of these parameters:

In [ ]:
p = 10
M = 150
beta = 1e-2
T = 473

The average area of a surface site is perhaps the less intuitive of the parameters. However, Machball provides two functions that can be used to calculate this value from the growth per cycle or the mass per cycle.

In [ ]:
from machball.utils import sitearea, sitearea_fromqcm

If we look at the documentation of these two functions:

In [ ]:
help(sitearea)

In [ ]:
help(sitearea_fromqcm)

They help establish a connection between experimental observables and the average area of a surface site. Let's use 35 ng/cm2 as the mass gain per cycle and the molar mass of Al2O3:

In [ ]:
sitearea_fromqcm(102, 35, 2)

Let's plot the value of the site area for alumina for a range of mass gain per cycle:

In [ ]:
mgain = np.arange(10,45,0.1)
areas = 1e18*sitearea_fromqcm(102, mgain, 2) # Convert to square Angstrom
pt.plot(mgain, areas)
pt.xlabel(r"Mass gain, ng/cm$^2$")
pt.ylabel(r"Surface site area, nm$^2$")

So let's define a value of s0:

In [ ]:
s0 = 20e-20

### Working widh ALDIdeal

Let's import ```ALDIdeal``` and define our ALD chemistry:

In [ ]:
from machball import ALDIdeal

ald = ALDIdeal(beta, M, T, p, s0, betarec=0)

The first thing we can do is to use the ```dose``` method to calculate the effect of a dose:

In [ ]:
ald.dose(0.02) # dose time in seconds

The returned value is the fractional coverage after the dose. It is important to remember that, at this point, doses are not accumulative yet. If you repeat the same dose, you will obtain the same value:

In [ ]:
ald.dose(0.02)

This is something that will be changed in the next version, to be able to monitor the effect of multiple microdoses

```ALDIdeal``` also has a method for directly calculating the saturation curve:


In [ ]:
times, cov = ald.saturation()

This method returns a list of times and saturation coverages. Let's plot them:

In [ ]:
pt.plot(1e3*times,cov)
pt.xlabel("Dose time, ms")
pt.ylabel("Fractional surface coverage")

We can save the saturation profile to a file using ```save_saturation```

In [ ]:
from machball.utils import save_saturation
save_saturation("mysaturation.dat", times, cov)

The default is to save it as a space-delimited file. If instead, you want to save it as a comma separated value, we need to specify the ```csv``` flag:

In [ ]:
save_saturation("mysaturation.csv", times, cov, csv=True)

In both cases the output can be imported pretty much anywhere. ```mysaturation.csv``` can be directly read by any spreadsheet software, should you use such unholy creatures.

## Simulating ballistic transport inside nanostructures

The first step is to define a nanostructure. For this tutorial, we are going to focus on circular vias, which are pre-defined in Machball. However, Machball allows for the simulation of ballistic transport in arbitrary geometries as long as the view factors are provided.

We first need to import and create our nanostructure:

In [ ]:
from machball.ballistic import Via

```Via``` discretizes a circular via in a number of equally sized segments plus the bottom. In order to instantiate a specific via, we need to pass the aspect ratio, defined as the depth/diameter ratio, and the number of vertical segments:

In [ ]:
st = Via(100, 200)

We can now use the same interface as we used for exploring the evolution of surface coverage on a flat surface. First, we can look at the result of a specific dose:

In [ ]:
cov = ald.dose(0.5, st)

The difference with the flat case is that we are passing the nanostructura as an argument to the ```dose``` method of our ```ALDIdeal``` object that we created before.

If we directly plot the output we see the coverage for all the different elements:

In [ ]:
pt.plot(cov)

It is important to note, though, that in the output of ```dose``` also contains the coverage in the bottom of the feature. In fact, if we look at the length of ```cov```:

In [ ]:
len(cov)

It has 201 elements whereas in the definition of our via we specified 200 vertical segments. The coverage at the bottom is stored in the last element of the array. Therefore, if we want to plot as a function of depth, we need to leave that element out:

In [ ]:
cov_wall = cov[:-1] # This creates a slice of the array which leaves the last element out

In [ ]:
len(cov_wall)

In [ ]:
pt.plot(cov_wall)

Like in the flat case, we can also directly calculate the saturation profile. This will give us a sequence of coverage profiles as a function of time:

In [ ]:
times2, covs2 = ald.saturation(st, endcov=0.95, verbose=False)

```covs2``` is a 2D array. If we look at its shape:

In [ ]:
covs2.shape

We see that each row represents a coverage profile for a given time.  

Using slices we can plot specific times:

In [ ]:
pt.plot(covs2[10,:-1], label=("%6.4f s" % times2[10]))
pt.plot(covs2[20,:-1], label=("%6.4f s" % times2[20]))
pt.plot(covs2[50,:-1], label=("%6.4f s" % times2[50]))
pt.legend()

Note that, as before, we have used  ```:-1``` to keep just the sidewall regions of our via. We can also view the 2D array as an image:

In [ ]:
pt.imshow(covs2[:,:-1])

However, it is important to note that the times are not linearly spaced, since Machball uses an adaptive timestep to evolve the fractional coverage with time:

In [ ]:
times2

We can also plot the average surface coverage as a function of time

In [ ]:
pt.plot(times2, np.mean(covs2,axis=1))
pt.xlabel("Dose time (s)")
pt.ylabel("Average surface coverage")

The resulting profiles has the parabolic shape that is expected from a transport-limited process.

Finally, we can export the coverage profiles and the times to a file:

In [ ]:
from machball.utils import save_saturationprofile

In [ ]:
save_saturationprofile("via100.dat", times2, covs2)

## Next steps

This tutorial just went over some of the more basic features of Machball. There are other capabilities we didn't go through that are fully documented, though not easily accessible. These include the ability to work with other types of eatures, to compare with constant reaction probability processes, and to expand the range of processes to incoporate surface recombination and soft-saturating reactions.